In [1]:
import scala.collection.Map
import scala.collection.mutable.ArrayBuffer
import scala.util.Random
import org.apache.spark.broadcast.Broadcast
import org.apache.spark.ml.recommendation.{ALS, ALSModel}
import org.apache.spark.sql.{DataFrame, Dataset, SparkSession}
import org.apache.spark.sql.functions._

Intitializing Scala interpreter ...

Spark Web UI available at http://912db3bbf5c4:4040
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1610966521758)
SparkSession available as 'spark'


import scala.collection.Map
import scala.collection.mutable.ArrayBuffer
import scala.util.Random
import org.apache.spark.broadcast.Broadcast
import org.apache.spark.ml.recommendation.{ALS, ALSModel}
import org.apache.spark.sql.{DataFrame, Dataset, SparkSession}
import org.apache.spark.sql.functions._


In [2]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:           1988        1023          69         398         894         444
Swap:          1023          23        1000



In [3]:
val spark = SparkSession.builder().config("spark.executor.memory", "4g").getOrCreate()
spark.sparkContext.getConf.getAll

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@2aac92f0
res0: Array[(String, String)] = Array((spark.driver.extraJavaOptions,"-Dio.netty.tryReflectionSetAccessible=true"), (spark.executor.id,driver), (spark.repl.class.uri,spark://912db3bbf5c4:46139/classes), (spark.driver.port,46139), (spark.driver.host,912db3bbf5c4), (spark.app.name,spylon-kernel), (spark.executor.extraJavaOptions,"-Dio.netty.tryReflectionSetAccessible=true"), (spark.rdd.compress,True), (spark.repl.class.outputDir,/tmp/tmpxjqlfwxg), (spark.app.id,local-1610966521758), (spark.serializer.objectStreamReset,100), (spark.master,local[*]), (spark.submit.pyFiles,""), (spark.submit.deployMode,client), (spark.ui.showConsoleProgress,true))


In [4]:
val base_path = "/home/jovyan/work/ch03/data/"
val user_artist_path = "user_artist_data.txt"
val artist_path = "artist_data.txt"
val alias_path = "artist_alias.txt"

base_path: String = /home/jovyan/work/ch03/data/
user_artist_path: String = user_artist_data.txt
artist_path: String = artist_data.txt
alias_path: String = artist_alias.txt


In [5]:
val rawUserArtistData = spark.read.textFile(base_path + user_artist_path)
rawUserArtistData.take(5).foreach(println)

1000002 1 55
1000002 1000006 33
1000002 1000007 8
1000002 1000009 144
1000002 1000010 314


rawUserArtistData: org.apache.spark.sql.Dataset[String] = [value: string]


In [6]:
rawUserArtistData.map{ line =>
    val Array(user, artist, _*) = line.split(' ')
    (user.toInt, artist.toInt)
}

res2: org.apache.spark.sql.Dataset[(Int, Int)] = [_1: int, _2: int]


In [7]:
val userArtistDF = rawUserArtistData.map{ line =>
    val Array(user, artist, _*) = line.split(' ')  // _도 괜찮던데?
    (user.toInt, artist.toInt)
}.toDF("user", "artist")

println(userArtistDF.count)

24296858


userArtistDF: org.apache.spark.sql.DataFrame = [user: int, artist: int]


In [8]:
userArtistDF.head(3)

res4: Array[org.apache.spark.sql.Row] = Array([1000002,1], [1000002,1000006], [1000002,1000007])


In [9]:
userArtistDF.limit(3)

res5: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [user: int, artist: int]


In [10]:
userArtistDF.show(3)

+-------+-------+
|   user| artist|
+-------+-------+
|1000002|      1|
|1000002|1000006|
|1000002|1000007|
+-------+-------+
only showing top 3 rows



In [11]:
userArtistDF.take(3)

res7: Array[org.apache.spark.sql.Row] = Array([1000002,1], [1000002,1000006], [1000002,1000007])


In [12]:
userArtistDF.agg(
    min("user"), max("user"), min("artist"), max("artist")
).show()

+---------+---------+-----------+-----------+
|min(user)|max(user)|min(artist)|max(artist)|
+---------+---------+-----------+-----------+
|       90|  2443548|          1|   10794401|
+---------+---------+-----------+-----------+



In [13]:
val b = userArtistDF.limit(1000000)
b.agg(
    min("user"), max("user"), min("artist"), max("artist")
).show()

+---------+---------+-----------+-----------+
|min(user)|max(user)|min(artist)|max(artist)|
+---------+---------+-----------+-----------+
|    10015|  1009847|          1|   10788645|
+---------+---------+-----------+-----------+



b: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [user: int, artist: int]


In [14]:
val rawArtistData = spark.read.textFile(base_path + artist_path)
rawArtistData.take(3).foreach(println)

1134999	06Crazy Life
6821360	Pang Nakarin
10113088	Terfel, Bartoli- Mozart: Don


rawArtistData: org.apache.spark.sql.Dataset[String] = [value: string]


rawArtistData.map { line =>
    val (id, name) = line.span(_ != '\t')
    (id.toInt, name.trim)
}.count()

In [15]:
val artistById = rawArtistData.flatMap { line =>
    val (id, name) = line.span(_ != '\t')
    if (name.isEmpty) {
        None
    } else {
        try {
            Some((id.toInt), name.trim)
        } catch {
            case _: NumberFormatException => None
        }
    }
}.toDF("id", "name")

artistById.take(2).foreach(println)

[1134999,06Crazy Life]
[6821360,Pang Nakarin]


artistById: org.apache.spark.sql.DataFrame = [id: int, name: string]


In [16]:
val rawArtistAlias = spark.read.textFile(base_path + alias_path)
rawArtistAlias.take(3).foreach(println)

1092764	1000311
1095122	1000557
6708070	1007267


rawArtistAlias: org.apache.spark.sql.Dataset[String] = [value: string]


In [17]:
val artistAlias = rawArtistAlias.flatMap{ line =>
    val Array(artist, alias) = line.split('\t')
    if (artist.isEmpty) {
        None
    } else {
        Some((artist.toInt, alias.toInt))
    }
}.collect().toMap

artistAlias: scala.collection.immutable.Map[Int,Int] = Map(1208690 -> 1003926, 2012757 -> 4569, 6949139 -> 1085752, 1109727 -> 1239120, 6772751 -> 1244705, 2070533 -> 1021544, 1157679 -> 2194, 9969617 -> 5630, 2034496 -> 1116214, 6764342 -> 40, 1272489 -> 1278238, 2108744 -> 1009267, 10349857 -> 1000052, 2145319 -> 1020463, 2126338 -> 2717, 10165456 -> 1001169, 6779368 -> 1239506, 10278137 -> 1001523, 9939075 -> 1329390, 2037201 -> 1274155, 1248585 -> 2885, 1106945 -> 1399, 6811322 -> 1019016, 9978396 -> 1784, 6676961 -> 1086433, 2117821 -> 2611, 6863616 -> 1277013, 6895480 -> 1000993, 6831632 -> 1246136, 1001719 -> 1009727, 10135633 -> 4250, 7029291 -> 1034635, 6967939 -> 1002734, 6864694 -> 1017311, 1237279 -> 1029752, 6793956 -> 1283231, 1208609 -> 1000699, 6693428 -> 1100258, 685174...


artistById.filter($"id" isin (1208690, 1003926)).show()

In [18]:
artistAlias.getOrElse(325235133, 0)

res13: Int = 0


In [19]:
def buildCounts(
    rawUserArtistData: Dataset[String],
    bArtistAlias: Broadcast[scala.collection.immutable.Map[Int, Int]]
) : DataFrame = {
    rawUserArtistData.map { line =>
        val Array(userID, artistID, count) = line.split(' ').map(_.toInt)
        val finalAritstID = bArtistAlias.value.getOrElse(artistID, artistID)
        (userID, finalAritstID, count)
    }.toDF("user", "artist", "count")
}

buildCounts: (rawUserArtistData: org.apache.spark.sql.Dataset[String], bArtistAlias: org.apache.spark.broadcast.Broadcast[scala.collection.immutable.Map[Int,Int]])org.apache.spark.sql.DataFrame


In [20]:
val sampleUserArtistData = rawUserArtistData.limit(1000000)

sampleUserArtistData: org.apache.spark.sql.Dataset[String] = [value: string]


val bArtistAlias = spark.sparkContext.broadcast(artistAlias)
val trainData = buildCounts(rawUserArtistData, bArtistAlias)
trainData.cache()

In [21]:
val bArtistAlias = spark.sparkContext.broadcast(artistAlias)
val trainData = buildCounts(sampleUserArtistData, bArtistAlias)
trainData.cache()

bArtistAlias: org.apache.spark.broadcast.Broadcast[scala.collection.immutable.Map[Int,Int]] = Broadcast(25)
trainData: org.apache.spark.sql.DataFrame = [user: int, artist: int ... 1 more field]
res14: trainData.type = [user: int, artist: int ... 1 more field]


In [22]:
trainData.take(1).foreach(println)

[1000002,1,55]


In [23]:
val model = new ALS().
  setSeed(Random.nextLong()).
  setImplicitPrefs(true).
  setRank(10).
  setRegParam(0.01).
  setAlpha(1.0).
  setMaxIter(5).
  setUserCol("user").
  setItemCol("artist").
  setRatingCol("count").
  setPredictionCol("prediction").
  fit(trainData)

model: org.apache.spark.ml.recommendation.ALSModel = ALSModel: uid=als_766fde51ac9c, rank=10


In [24]:
model.userFactors.show(2, truncate=false)

+-------+-------------------------------------------------------------------------------------------------------------------------------------+
|id     |features                                                                                                                             |
+-------+-------------------------------------------------------------------------------------------------------------------------------------+
|1000020|[-0.09962447, -0.013246033, 0.5240463, -0.08857838, 0.56820846, -0.6588475, -0.149682, -0.8199164, -1.3176633, -0.17163086]          |
|1000030|[-0.0069780033, 0.022746379, -0.02518704, -0.026718441, 0.04769788, 0.025246091, 0.06673875, 0.013707696, -0.057255972, -0.035054445]|
+-------+-------------------------------------------------------------------------------------------------------------------------------------+
only showing top 2 rows



In [25]:
val userID = 1000020
val existingArtisdIDs = trainData.filter($"user" === userID).select("artist").as[Int].collect()
artistById.filter($"id" isin (existingArtisdIDs:_*)).show()

+-------+--------------------+
|     id|                name|
+-------+--------------------+
|1248399|   Bjrk & Thom Yorke|
|1251278|     Love Of Lesbian|
|1004461|          Gary Jules|
|1252420|   The Ordinary Boys|
|1262123|Valley of the Giants|
|1268554|Elvis Presley vs....|
|   5496| Echo & the Bunnymen|
|1001891|Elvis Costello & ...|
|   1195|      Elvis Costello|
|1281772|       King Creosote|
|1004513|     Daniel Johnston|
|   5795|            Manitoba|
|   4985|            Hangedup|
|   1870|           808 State|
|1001265|            Elastica|
|1014826|       Junior Senior|
|   5687|The Jimi Hendrix ...|
|1001286|                Gene|
|1000848|          Roxy Music|
|1029443|Meanwhile, Back I...|
+-------+--------------------+
only showing top 20 rows



userID: Int = 1000020
existingArtisdIDs: Array[Int] = Array(1, 1000024, 1000028, 1000044, 1000054, 1000062, 1000105, 1000113, 1000127, 1000130, 1000139, 1000188, 1000230, 1000233, 1000283, 1000289, 1000313, 1000315, 1000323, 1000333, 1000343, 1000388, 1000421, 1000429, 1000462, 1000505, 1000522, 1000557, 1000569, 1000591, 1000631, 1000660, 1000665, 1000676, 1000722, 1000724, 1000846, 1000848, 1000860, 1000864, 1000886, 1000930, 1000977, 1000985, 1001002, 15, 1001062, 1001072, 1001134, 1001145, 1001148, 1001152, 1001169, 1001265, 1001278, 1001286, 1001301, 1001306, 1001363, 1001383, 1001428, 1001435, 1001443, 1001449, 1001450, 1001459, 1001514, 1001525, 1001531, 1001588, 1001592, 1001646, 1001713, 1001714, 1001727, 1001735, 1001741, 1001762, 1001770, 1001779, 1001822, 1001864, 1268554, 1...


In [26]:
def makeRecommendations(
    model: ALSModel,
    userID: Int,
    howMany: Int): DataFrame = {
    
    val toRecommend = model.itemFactors.select($"id".as("artist")).withColumn("user", lit(userID))
    model.transform(toRecommend).select("artist", "prediction").orderBy($"prediction".desc).limit(howMany)
}

makeRecommendations: (model: org.apache.spark.ml.recommendation.ALSModel, userID: Int, howMany: Int)org.apache.spark.sql.DataFrame


In [27]:
val toRecommend = model.itemFactors.select($"id".as("artist")).withColumn("user", lit(1000020))
toRecommend.show(3)

+------+-------+
|artist|   user|
+------+-------+
|    30|1000020|
|    40|1000020|
|    70|1000020|
+------+-------+
only showing top 3 rows



toRecommend: org.apache.spark.sql.DataFrame = [artist: int, user: int]


In [28]:
lit(100)

res19: org.apache.spark.sql.Column = 100


In [29]:
val topRecommendations = makeRecommendations(model, userID, 5)
topRecommendations.show(5)

+-------+----------+
| artist|prediction|
+-------+----------+
|   1235| 1.1371856|
|   1187| 1.1039873|
|   1262| 1.0945467|
|    232| 1.0836058|
|1034635| 1.0827067|
+-------+----------+



topRecommendations: org.apache.spark.sql.DataFrame = [artist: int, prediction: float]


In [30]:
val recoArtistIDs = topRecommendations.select("artist").as[Int].collect()
artistById.filter($"id" isin (recoArtistIDs:_*)).show()

+-------+--------------------+
|     id|                name|
+-------+--------------------+
|1034635|           [unknown]|
|    232|                Pulp|
|   1262|           PJ Harvey|
|   1235|Kings of Convenience|
|   1187|               Doves|
+-------+--------------------+



recoArtistIDs: Array[Int] = Array(1235, 1187, 1262, 232, 1034635)


- **AUC, Hyperparemeter 제외**

In [31]:
def areaUnderCurve(
      positiveData: DataFrame,
      bAllArtistIDs: Broadcast[Array[Int]],
      predictFunction: (DataFrame => DataFrame)): Double = {

    // What this actually computes is AUC, per user. The result is actually something
    // that might be called "mean AUC".

    // Take held-out data as the "positive".
    // Make predictions for each of them, including a numeric score
    val positivePredictions = predictFunction(positiveData.select("user", "artist")).
      withColumnRenamed("prediction", "positivePrediction")

    // BinaryClassificationMetrics.areaUnderROC is not used here since there are really lots of
    // small AUC problems, and it would be inefficient, when a direct computation is available.

    // Create a set of "negative" products for each user. These are randomly chosen
    // from among all of the other artists, excluding those that are "positive" for the user.
    val negativeData = positiveData.select("user", "artist").as[(Int,Int)].
      groupByKey { case (user, _) => user }.
      flatMapGroups { case (userID, userIDAndPosArtistIDs) =>
        val random = new Random()
        val posItemIDSet = userIDAndPosArtistIDs.map { case (_, artist) => artist }.toSet
        val negative = new ArrayBuffer[Int]()
        val allArtistIDs = bAllArtistIDs.value
        var i = 0
        // Make at most one pass over all artists to avoid an infinite loop.
        // Also stop when number of negative equals positive set size
        while (i < allArtistIDs.length && negative.size < posItemIDSet.size) {
          val artistID = allArtistIDs(random.nextInt(allArtistIDs.length))
          // Only add new distinct IDs
          if (!posItemIDSet.contains(artistID)) {
            negative += artistID
          }
          i += 1
        }
        // Return the set with user ID added back
        negative.map(artistID => (userID, artistID))
      }.toDF("user", "artist")

    // Make predictions on the rest:
    val negativePredictions = predictFunction(negativeData).
      withColumnRenamed("prediction", "negativePrediction")

    // Join positive predictions to negative predictions by user, only.
    // This will result in a row for every possible pairing of positive and negative
    // predictions within each user.
    val joinedPredictions = positivePredictions.join(negativePredictions, "user").
      select("user", "positivePrediction", "negativePrediction").cache()

    // Count the number of pairs per user
    val allCounts = joinedPredictions.
      groupBy("user").agg(count(lit("1")).as("total")).
      select("user", "total")
    // Count the number of correctly ordered pairs per user
    val correctCounts = joinedPredictions.
      filter($"positivePrediction" > $"negativePrediction").
      groupBy("user").agg(count("user").as("correct")).
      select("user", "correct")

    // Combine these, compute their ratio, and average over all users
    val meanAUC = allCounts.join(correctCounts, "user").
      select($"user", ($"correct" / $"total").as("auc")).
      agg(mean("auc")).
      as[Double].first()

    joinedPredictions.unpersist()

    meanAUC
    }

areaUnderCurve: (positiveData: org.apache.spark.sql.DataFrame, bAllArtistIDs: org.apache.spark.broadcast.Broadcast[Array[Int]], predictFunction: org.apache.spark.sql.DataFrame => org.apache.spark.sql.DataFrame)Double


In [32]:
val allData = buildCounts(rawUserArtistData, bArtistAlias)
val someUsers = allData.select("user").as[Int].distinct().take(10)

allData: org.apache.spark.sql.DataFrame = [user: int, artist: int ... 1 more field]
someUsers: Array[Int] = Array(1000190, 1001043, 1001129, 1001139, 1002431, 1002605, 1004666, 1005158, 1005439, 1005697)


In [33]:
val someRecommendations = someUsers.map(userID => (userID, makeRecommendations(model, userID, 5)))
someRecommendations.foreach { case (userID, recsDF) => 
    val recommendedArtist = recsDF.select("artist").as[Int].collect()
    println(s"$userID -> ${recommendedArtist.mkString(", ")}")
}

1000190 -> 6694932, 1006411, 1233321, 420, 1006354
1001043 -> 1854, 393, 1006016, 930, 1034635
1001129 -> 393, 1034635, 15, 59, 979
1001139 -> 393, 59, 1034635, 15, 121
1002431 -> 4629, 3327, 1001646, 1274, 1000062
1002605 -> 393, 4629, 1034635, 930, 1274
1004666 -> 1000442, 1058104, 4629, 1002560, 1034635
1005158 -> 1854, 4468, 1006016, 4629, 1000052
1005439 -> 1000183, 1001384, 1000152, 1000630, 1002280
1005697 -> 1002065, 2814, 1854, 930, 1177


someRecommendations: Array[(Int, org.apache.spark.sql.DataFrame)] = Array((1000190,[artist: int, prediction: float]), (1001043,[artist: int, prediction: float]), (1001129,[artist: int, prediction: float]), (1001139,[artist: int, prediction: float]), (1002431,[artist: int, prediction: float]), (1002605,[artist: int, prediction: float]), (1004666,[artist: int, prediction: float]), (1005158,[artist: int, prediction: float]), (1005439,[artist: int, prediction: float]), (1005697,[artist: int, prediction: float]))


In [ ]:
class RunRecommender(private val spark: SparkSession) {


  def model(
      rawUserArtistData: Dataset[String],
      rawArtistData: Dataset[String],
      rawArtistAlias: Dataset[String]): Unit = {

    val bArtistAlias = spark.sparkContext.broadcast(buildArtistAlias(rawArtistAlias))

    val trainData = buildCounts(rawUserArtistData, bArtistAlias).cache()

    val model = new ALS().
      setSeed(Random.nextLong()).
      setImplicitPrefs(true).
      setRank(10).
      setRegParam(0.01).
      setAlpha(1.0).
      setMaxIter(5).
      setUserCol("user").
      setItemCol("artist").
      setRatingCol("count").
      setPredictionCol("prediction").
      fit(trainData)

    trainData.unpersist()

    model.userFactors.select("features").show(truncate = false)

    val userID = 2093760

    val existingArtistIDs = trainData.
      filter($"user" === userID).
      select("artist").as[Int].collect()

    val artistByID = buildArtistByID(rawArtistData)

    artistByID.filter($"id" isin (existingArtistIDs:_*)).show()

    val topRecommendations = makeRecommendations(model, userID, 5)
    topRecommendations.show()

    val recommendedArtistIDs = topRecommendations.select("artist").as[Int].collect()

    artistByID.filter($"id" isin (recommendedArtistIDs:_*)).show()

    model.userFactors.unpersist()
    model.itemFactors.unpersist()
  }

  def evaluate(
      rawUserArtistData: Dataset[String],
      rawArtistAlias: Dataset[String]): Unit = {

    val bArtistAlias = spark.sparkContext.broadcast(buildArtistAlias(rawArtistAlias))

    val allData = buildCounts(rawUserArtistData, bArtistAlias)
    val Array(trainData, cvData) = allData.randomSplit(Array(0.9, 0.1))
    trainData.cache()
    cvData.cache()

    val allArtistIDs = allData.select("artist").as[Int].distinct().collect()
    val bAllArtistIDs = spark.sparkContext.broadcast(allArtistIDs)

    val mostListenedAUC = areaUnderCurve(cvData, bAllArtistIDs, predictMostListened(trainData))
    println(mostListenedAUC)

    val evaluations =
      for (rank     <- Seq(5,  30);
           regParam <- Seq(1.0, 0.0001);
           alpha    <- Seq(1.0, 40.0))
      yield {
        val model = new ALS().
          setSeed(Random.nextLong()).
          setImplicitPrefs(true).
          setRank(rank).setRegParam(regParam).
          setAlpha(alpha).setMaxIter(20).
          setUserCol("user").setItemCol("artist").
          setRatingCol("count").setPredictionCol("prediction").
          fit(trainData)

        val auc = areaUnderCurve(cvData, bAllArtistIDs, model.transform)

        model.userFactors.unpersist()
        model.itemFactors.unpersist()

        (auc, (rank, regParam, alpha))
      }

    evaluations.sorted.reverse.foreach(println)

    trainData.unpersist()
    cvData.unpersist()
  }

  def recommend(
      rawUserArtistData: Dataset[String],
      rawArtistData: Dataset[String],
      rawArtistAlias: Dataset[String]): Unit = {

    val bArtistAlias = spark.sparkContext.broadcast(buildArtistAlias(rawArtistAlias))
    val allData = buildCounts(rawUserArtistData, bArtistAlias).cache()
    val model = new ALS().
      setSeed(Random.nextLong()).
      setImplicitPrefs(true).
      setRank(10).setRegParam(1.0).setAlpha(40.0).setMaxIter(20).
      setUserCol("user").setItemCol("artist").
      setRatingCol("count").setPredictionCol("prediction").
      fit(allData)
    allData.unpersist()

    val userID = 2093760
    val topRecommendations = makeRecommendations(model, userID, 5)

    val recommendedArtistIDs = topRecommendations.select("artist").as[Int].collect()
    val artistByID = buildArtistByID(rawArtistData)
    artistByID.join(spark.createDataset(recommendedArtistIDs).toDF("id"), "id").
      select("name").show()

    model.userFactors.unpersist()
    model.itemFactors.unpersist()
  }

  def buildArtistByID(rawArtistData: Dataset[String]): DataFrame = {
    rawArtistData.flatMap { line =>
      val (id, name) = line.span(_ != '\t')
      if (name.isEmpty) {
        None
      } else {
        try {
          Some((id.toInt, name.trim))
        } catch {
          case _: NumberFormatException => None
        }
      }
    }.toDF("id", "name")
  }

  def buildArtistAlias(rawArtistAlias: Dataset[String]): Map[Int,Int] = {
    rawArtistAlias.flatMap { line =>
      val Array(artist, alias) = line.split('\t')
      if (artist.isEmpty) {
        None
      } else {
        Some((artist.toInt, alias.toInt))
      }
    }.collect().toMap
  }

  def buildCounts(
      rawUserArtistData: Dataset[String],
      bArtistAlias: Broadcast[Map[Int,Int]]): DataFrame = {
    rawUserArtistData.map { line =>
      val Array(userID, artistID, count) = line.split(' ').map(_.toInt)
      val finalArtistID = bArtistAlias.value.getOrElse(artistID, artistID)
      (userID, finalArtistID, count)
    }.toDF("user", "artist", "count")
  }

  def makeRecommendations(model: ALSModel, userID: Int, howMany: Int): DataFrame = {
    val toRecommend = model.itemFactors.
      select($"id".as("artist")).
      withColumn("user", lit(userID))
    model.transform(toRecommend).
      select("artist", "prediction").
      orderBy($"prediction".desc).
      limit(howMany)
  }

  def areaUnderCurve(
      positiveData: DataFrame,
      bAllArtistIDs: Broadcast[Array[Int]],
      predictFunction: (DataFrame => DataFrame)): Double = {

    // What this actually computes is AUC, per user. The result is actually something
    // that might be called "mean AUC".

    // Take held-out data as the "positive".
    // Make predictions for each of them, including a numeric score
    val positivePredictions = predictFunction(positiveData.select("user", "artist")).
      withColumnRenamed("prediction", "positivePrediction")

    // BinaryClassificationMetrics.areaUnderROC is not used here since there are really lots of
    // small AUC problems, and it would be inefficient, when a direct computation is available.

    // Create a set of "negative" products for each user. These are randomly chosen
    // from among all of the other artists, excluding those that are "positive" for the user.
    val negativeData = positiveData.select("user", "artist").as[(Int,Int)].
      groupByKey { case (user, _) => user }.
      flatMapGroups { case (userID, userIDAndPosArtistIDs) =>
        val random = new Random()
        val posItemIDSet = userIDAndPosArtistIDs.map { case (_, artist) => artist }.toSet
        val negative = new ArrayBuffer[Int]()
        val allArtistIDs = bAllArtistIDs.value
        var i = 0
        // Make at most one pass over all artists to avoid an infinite loop.
        // Also stop when number of negative equals positive set size
        while (i < allArtistIDs.length && negative.size < posItemIDSet.size) {
          val artistID = allArtistIDs(random.nextInt(allArtistIDs.length))
          // Only add new distinct IDs
          if (!posItemIDSet.contains(artistID)) {
            negative += artistID
          }
          i += 1
        }
        // Return the set with user ID added back
        negative.map(artistID => (userID, artistID))
      }.toDF("user", "artist")

    // Make predictions on the rest:
    val negativePredictions = predictFunction(negativeData).
      withColumnRenamed("prediction", "negativePrediction")

    // Join positive predictions to negative predictions by user, only.
    // This will result in a row for every possible pairing of positive and negative
    // predictions within each user.
    val joinedPredictions = positivePredictions.join(negativePredictions, "user").
      select("user", "positivePrediction", "negativePrediction").cache()

    // Count the number of pairs per user
    val allCounts = joinedPredictions.
      groupBy("user").agg(count(lit("1")).as("total")).
      select("user", "total")
    // Count the number of correctly ordered pairs per user
    val correctCounts = joinedPredictions.
      filter($"positivePrediction" > $"negativePrediction").
      groupBy("user").agg(count("user").as("correct")).
      select("user", "correct")

    // Combine these, compute their ratio, and average over all users
    val meanAUC = allCounts.join(correctCounts, "user").
      select($"user", ($"correct" / $"total").as("auc")).
      agg(mean("auc")).
      as[Double].first()

    joinedPredictions.unpersist()

    meanAUC
  }

  def predictMostListened(train: DataFrame)(allData: DataFrame): DataFrame = {
    val listenCounts = train.groupBy("artist").
      agg(sum("count").as("prediction")).
      select("artist", "prediction")
    allData.
      join(listenCounts, Seq("artist"), "left_outer").
      select("user", "artist", "prediction")
  }

}

In [ ]:
object RunRecommender {

  def main(args: Array[String]): Unit = {
    val spark = SparkSession.builder().getOrCreate()
    // Optional, but may help avoid errors due to long lineage
    spark.sparkContext.setCheckpointDir("hdfs:///tmp/")

    val base = "hdfs:///user/ds/"
    val rawUserArtistData = spark.read.textFile(base + "user_artist_data.txt")
    val rawArtistData = spark.read.textFile(base + "artist_data.txt")
    val rawArtistAlias = spark.read.textFile(base + "artist_alias.txt")

    val runRecommender = new RunRecommender(spark)
    runRecommender.preparation(rawUserArtistData, rawArtistData, rawArtistAlias)
    runRecommender.model(rawUserArtistData, rawArtistData, rawArtistAlias)
    runRecommender.evaluate(rawUserArtistData, rawArtistAlias)
    runRecommender.recommend(rawUserArtistData, rawArtistData, rawArtistAlias)
  }

}